<a href="https://colab.research.google.com/github/soumyaunique/summeranalytics2025/blob/week-1/Dynamic_Pricing_for_Urban_Parking_Spots.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Inroduction


This sample notebook demonstrates how to process live data streams using Pathway. The dataset used here is a subset of the one provided — specifically, it includes data for only a single parking spot. You are expected to implement your model across all parking spots.

Please note that the pricing model used in this notebook is a simple baseline. You are expected to design and implement a more advanced and effective model.


In [2]:
!pip install pathway bokeh --quiet # This cell may take a few seconds to execute.

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.6/777.6 kB 41.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 71.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.6/244.6 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.4/318.4 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 985.8/985.8 kB 45.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from datetime import datetime
import pathway as pw
import bokeh.plotting
import panel as pn

# Step 1: Importing and Preprocessing the Data

In [6]:
df = pd.read_csv('/tmp/data/dataset.csv')
df

# You can find the sample dataset here: https://drive.google.com/file/d/1D479FLjp9aO3Mg8g6Lpj9oRViWacurA6/view?usp=sharing

,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime
0,0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,04-10-2016,07:59:00
1,1,BHMBCCMKT01,577,26.144536,91.736172,64,car,low,1,0,04-10-2016,08:25:00
2,2,BHMBCCMKT01,577,26.144536,91.736172,80,car,low,2,0,04-10-2016,08:59:00
3,3,BHMBCCMKT01,577,26.144536,91.736172,107,car,low,2,0,04-10-2016,09:32:00
4,4,BHMBCCMKT01,577,26.144536,91.736172,150,bike,low,2,0,04-10-2016,09:59:00
...,...,...,...,...,...,...,...,...,...,...,...,...
18363,18363,Shopping,1920,26.150504,91.733531,1517,truck,average,6,0,19-12-2016,14:30:00
18364,18364,Shopping,1920,26.150504,91.733531,1487,car,low,3,0,19-12-2016,15:03:00
18365,18365,Shopping,1920,26.150504,91.733531,1432,cycle,low,3,0,19-12-2016,15:29:00
18366,18366,Shopping,1920,26.150504,91.733531,1321,car,low,2,0,19-12-2016,16:03:00


In [7]:
# Step 3: Preprocessing
# Map VehicleType and TrafficConditionNearby to numeric
vehicle_type_map = {'car': 2.0, 'bike': 1.5, 'truck': 2.5, 'cycle':1.0}
traffic_map = {'low': 0, 'average': 1, 'high': 2}

df['VehicleTypeWeight'] = df['VehicleType'].map(vehicle_type_map)
df['TrafficLevel'] = df['TrafficConditionNearby'].map(traffic_map)

In [8]:
# Step 4: Baseline Pricing Model
def baseline_price(prev_price, occupancy, capacity, alpha=0.5):
    occupancy_rate = occupancy / capacity
    return prev_price + alpha * occupancy_rate

# Example usage:
df['BaselinePrice'] = 10  # Start with $10 base price
for i in range(1, len(df)):
    df.loc[i, 'BaselinePrice'] = baseline_price(
        df.loc[i - 1, 'BaselinePrice'],
        df.loc[i, 'Occupancy'],
        df.loc[i, 'Capacity']
    )

/tmp/ipython-input-8-2375515439.py:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '10.055459272097053' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[i, 'BaselinePrice'] = baseline_price(


In [9]:
# Step 5: Demand-Based Pricing Model
def calculate_demand(row, alpha=0.6, beta=0.4, gamma=0.3, delta=0.5, epsilon=0.2):
    occ_rate = row['Occupancy'] / row['Capacity']
    return (
        alpha * occ_rate +
        beta * row['QueueLength'] -
        gamma * row['TrafficLevel'] +
        delta * row['IsSpecialDay'] +
        epsilon * row['VehicleTypeWeight']
    )

def demand_based_price(base_price, demand, lambda_=0.8, min_factor=0.5, max_factor=2.0):
    norm_demand = np.clip(demand / 10, -1, 1)
    price = base_price * (1 + lambda_ * norm_demand)
    return np.clip(price, base_price * min_factor, base_price * max_factor)

# Apply demand-based pricing
df['Demand'] = df.apply(calculate_demand, axis=1)
df['DemandBasedPrice'] = df['Demand'].apply(lambda d: demand_based_price(10, d))


In [10]:
from bokeh.plotting import figure, show, output_notebook

In [11]:
# Step 7: Visualization (Example for Baseline Pricing)
p = figure(title="Baseline Pricing Over Time", x_axis_label='Time Step', y_axis_label='Price')
p.line(df.index, df['BaselinePrice'], line_width=2)
show(p)

In [10]:
p

figure(id='51e66941-ef33-4198-acfe-82fffa51fb13', ...)

In [12]:
df.to_csv("/tmp/data/pricing_output.csv", index=False)

In [13]:
# Combine the 'LastUpdatedDate' and 'LastUpdatedTime' columns into a single datetime column
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],
                                  format='%d-%m-%Y %H:%M:%S')

# Sort the DataFrame by the new 'Timestamp' column and reset the index
df = df.sort_values('Timestamp').reset_index(drop=True)

In [14]:
# Save the selected columns to a CSV file for streaming or downstream processing

# Ensure the 'DemandBasedPrice' column is of type float32
df['DemandBasedPrice'] = df['DemandBasedPrice'].astype(np.float32)

df[["Timestamp", "Occupancy", "Capacity","SystemCodeNumber", "DemandBasedPrice"]].to_csv("pricing_output.csv", index=False)

# Note: Only three features are used here for simplicity.
# Participants are expected to incorporate additional relevant features in their models.

In [15]:
# Define the schema for the streaming data using Pathway
# This schema specifies the expected structure of each data row in the stream

class ParkingSchema(pw.Schema):
    Timestamp: str   # Timestamp of the observation (should ideally be in ISO format)
    Occupancy: int   # Number of occupied parking spots
    Capacity: int    # Total parking capacity at the location
    SystemCodeNumber: str #Parking Space Identifier
    DemandBasedPrice: np.float32 #Price based on Demand


In [16]:
# Load the data as a simulated stream using Pathway's replay_csv function
# This replays the CSV data at a controlled input rate to mimic real-time streaming
# input_rate=1000 means approximately 1000 rows per second will be ingested into the stream.

data = pw.demo.replay_csv("pricing_output.csv", schema=ParkingSchema, input_rate=100)

In [17]:
# Define the datetime format to parse the 'Timestamp' column
fmt = "%Y-%m-%d %H:%M:%S"

# Add new columns to the data stream:
# - 't' contains the parsed full datetime
# - 'day' extracts the date part and resets the time to midnight (useful for day-level aggregations)
data_with_time = data.with_columns(
    t = data.Timestamp.dt.strptime(fmt),
    day = data.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00"),
    #system= data.SystemCodeNumber
)


# Step 2: Making a simple pricing function

In [18]:
# Define a daily tumbling window over the data stream using Pathway
# This block performs temporal aggregation and computes a dynamic price for each day
import datetime

delta_window = (
    data_with_time.windowby(
        pw.this.t,  # Event time column to use for windowing (parsed datetime)
        #instance=pw.this.day,  # Logical partitioning key: one instance per calendar day
        instance=pw.this.SystemCodeNumber,
        window=pw.temporal.tumbling(datetime.timedelta(days=1)),  # Fixed-size daily window
        behavior=pw.temporal.exactly_once_behavior()  # Guarantees exactly-once processing semantics
    )
    .reduce(
        SystemCodeNumber=pw.reducers.min(pw.this.SystemCodeNumber),
        t=(pw.this._pw_window_end), # Assign and explicitly cast the end timestamp of each window to datetime
        occ_max=pw.reducers.max(pw.this.Occupancy),      # Highest occupancy observed in the window
        occ_min=pw.reducers.min(pw.this.Occupancy),      # Lowest occupancy observed in the window
        cap=pw.reducers.max(pw.this.Capacity), # Maximum capacity observed (typically constant per spot)
        pricedemand=pw.reducers.max(pw.this.DemandBasedPrice)
    )
    .with_columns(
        # Compute the price using a simple dynamic pricing formula:
        #
        # Pricing Formula:
        #     price = base_price + demand_fluctuation
        #     where:
        #         base_price = 10 (fixed minimum price)
        #         demand_fluctuation = (occ_max - occ_min) / cap
        #
        # Intuition:
        # - The greater the difference between peak and low occupancy in a day,
        #   the more volatile the demand is, indicating potential scarcity.
        # - Dividing by capacity normalizes the fluctuation (to stay in [0,1] range).
        # - This fluctuation is added to the base price of 10 to set the final price.
        # - Example: If occ_max = 90, occ_min = 30, cap = 100
        #            => price = 10 + (90 - 30)/100 = 10 + 0.6 = 10.6

        #price=10 + (pw.this.occ_max - pw.this.occ_min) / pw.this.cap
        price=pw.cast(float, pw.this.pricedemand)# Explicitly cast price to float
    )
)

# Step 3: Visualizing Daily Price Fluctuations with a Bokeh Plot

**Note:** The Bokeh plot in the next cell will only be generated after you run the `pw.run()` cell (i.e., the final cell).


In [19]:
# Activate the Panel extension to enable interactive visualizations
pn.extension()

# Define a custom Bokeh plotting function that takes a data source (from Pathway) and returns a figure
def price_plotter(source):
    # Create a Bokeh figure with datetime x-axis
    fig = bokeh.plotting.figure(
        height=400,
        width=800,
        title="Pathway: Daily Parking Price",
        x_axis_type="datetime",  # Ensure time-based data is properly formatted on the x-axis
    )
    # Plot a line graph showing how the price evolves over time
    fig.line("t", "price", source=source, line_width=2, color="navy")

    # Overlay red circles at each data point for better visibility
    fig.circle("t", "price", source=source, size=6, color="red")

    return fig

# Custom Bokeh plotting function for multiple parking spaces
def multi_parking_plotter(source):
    fig = bokeh.plotting.figure(
        height=500,
        width=900,
        title="Real-Time Pricing per Parking Lot",
        x_axis_type="datetime",
    )

    # Extract unique SystemCodeNumbers from the data source's 'SystemCodeNumber' column
    # Assume that 'SystemCodeNumber' is already included in the data passed by Pathway
    # Extract unique SystemCodeNumbers
    system_codes = sorted(set(source.data['SystemCodeNumber']))

    # Color palette
    palette = bokeh.palettes.Category20[20] + bokeh.palettes.Category10[10]

    # Plot a line for the entire dataset (optional - still navy)
    fig.line("t", "price", source=source, line_width=2, color="navy")

    # Plot colored circles for each SystemCodeNumber
    for i, sys_code in enumerate(system_codes):
        # Filter the source data
        filtered_data = {
            't': [t for t, s in zip(source.data['t'], source.data['SystemCodeNumber']) if s == sys_code],
            'price': [p for p, s in zip(source.data['price'], source.data['SystemCodeNumber']) if s == sys_code],
        }

        filtered_source = bokeh.models.ColumnDataSource(filtered_data)

        # Add circles in a unique color
        fig.circle("t", "price", source=filtered_source, size=6, color=palette[i % len(palette)], legend_label=sys_code)

    # Make the legend interactive
    fig.legend.click_policy = "hide"
    return fig

# Use Pathway's built-in .plot() method to bind the data stream (delta_window) to the Bokeh plot
# - 'price_plotter' is the rendering function
# - 'sorting_col="t"' ensures the data is plotted in time order
viz = delta_window.plot(price_plotter, sorting_col="t")
#viz = delta_window.plot(multi_parking_plotter, sorting_col="t")

# Create a Panel layout and make it servable as a web app
# This line enables the interactive plot to be displayed when the app is served
pn.Column(viz).servable()

Column
    [0] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)

In [20]:
# Start the Pathway pipeline execution in the background
# - This triggers the real-time data stream processing defined above
# - %%capture --no-display suppresses output in the notebook interface

%%capture --no-display
pw.run()

Output()